In [167]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import unicodedata
import csv
import sys

In [193]:
class Get_Player_info() :
    
    def get_info(self,search_word) :
        data = requests.get(r"http://stats.espncricinfo.com/ci/engine/stats/analysis.html?search="+search_word)
        data_html = BeautifulSoup(data.text)
        
        #Get all player name
        name = data_html.find_all('span',{'style':"white-space: nowrap"})
        
        #Get all player numbers
        s = ''
        numbers = " ".join([x['href'] for x in data_html.findAll('a',{"class":"statsLinks"}) ])
        player_num = (re.findall("/player/(\d+)\.html?",numbers))
        
        #remove duplicates and preserve order
        player_num_new = []
        for x in player_num :
            if x not in player_num_new :
                player_num_new.append(x)
        
        #"name" will have two names for each player
        #RR David (Ramesh David)
        #S Ramesh (Sadagoppan Ramesh)
        # So we make logic to combine two names and then map it with correct number
        player_info = {}
        for i,a in enumerate(name) :
            if i%2 == 0 :
                num_name = a.text
            else :
                num_name = num_name + ' ' + a.text
                player_info[num_name] = player_num_new[i/2]
        
        return player_info
        
        
    def get_ALL_player_info(self,format_class) :
        class_num = 1
        if format_class.lower() == 'test' :
            class_num = 1
        elif format_class.lower() == 'odi' :
            class_num = 2 
        elif format_class.lower() == 't20' :
            class_num = 3
        else :
            sys.exit("Incorrect Format !!")
            
        ALL_player_dict = {}
        print "Please have some coffee till script gets you all the player info.. patience please ;)" 
        #Request the website for each of these numbers and get the corresponding number
        for i in range(1000,40000) :
            data = requests.get(r"http://stats.espncricinfo.com/ci/engine/player/"+str(i)+".html?class="+str(class_num)+";type=allround")
            data_html = BeautifulSoup(data.text)
            name = data_html.find('h1',{'class':"SubnavSitesection"})
            if name :
                print str(i)
                player_name = name.text.split('\n')[4]
                player_name = unicodedata.normalize('NFKD', player_name).encode('ascii','ignore')
                player_name = player_name.strip('/').strip(' ')
                print "Caught "+player_name+" at  "+str(i)
                ALL_player_dict[player_name] = i
        
        return ALL_player_dict
    
    def write_to_csv(self,path,player_dict) :
        with open(path, 'wb') as f:
            w = csv.writer(f)
            for x in player_dict :
                w.writerow((x,player_dict[x]))
        

In [194]:
obj = Get_Player_info()

In [196]:
obj.get_info('ramesh')

{u'(Ramesh Saxena) RK Sethi': '33882',
 u'(Ramesh Sethi) RB Shah': '23972',
 u'(Ramesh Shah) RD Singh': '47180',
 u'M Ramesh Kumar (Ramesh Kumar)': '391441',
 u'R Jadhav (Ramesh Jadhav)': '29643',
 u'RR David (Ramesh David)': '365126',
 u'RR Powar (Ramesh Powar)': '32391',
 u'RTM Mendis (Ramesh Mendis)': '629084',
 u'RV Divecha (Ramesh Divecha)': '28095',
 u'Ramesh Menon RC Saxena': '522800',
 u'S Ramesh (Sadagoppan Ramesh)': '33066',
 u'S Ramesh (Srinivasan Ramesh)': '33415'}

In [ ]:
player_dict = obj.get_ALL_player_info(format_class = "odi")

In [ ]:
obj.write_to_csv("player_info.csv",player_dict)